In [1]:
import pandas as pd
import numpy as np

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow.keras import layers


In [131]:
SENTIMENT_TEST = [
    ["Board Member Gilmore stated a nasty letter should be sent along with the check.", 0],
    ["Mayor Johnson stated finding money for City parks is very difficult.", 0],
    ["Even if you are frustrated by the recommendation, it is poor form to attack the presenter.", 0],
    ["Councilmember Daysog stated that he opposes putting a measure on the ballot because the City has the wrong reserve policy in place; a lot of money that that has gone towards pensions could have gone towards infrastructure.", 0],
    ["Opponent: (Not in favor of project appeal): Richard W. Rutter Alameda.", 0],
    ["Stated the intersection is very dangerous and moving the bus stop would not help: Demeter Lamb, Alameda.", 0],
    ["Not happy, as well as other residents too.", 0],
    ["Our vacation rental loses money.", 0],
    [" 	Ms. Farrar stated the name selected by the committee is not appropriate for a school.", 0],
    ["The Sabankayas’ have been dishonest in the past (scheduling weddings after October 2013-the Planning dept.", 0],
    ["Stated the intersection is very dangerous and moving the bus stop would not help: Demeter Lamb, Alameda.", 0],
    ["There is a serious lack of attention to community needs.", 0],
    ["Councilmember Tam stated Council’s adopted policy of having a 20%%-25%% fund balance is meaningless.", 0],
    ["Dorothy Freeman said the new project will block the views of the estuary even more than the current “brown wall” on Clement Ave.", 0],
    ["Despite our best efforts to ensure that only essential transcripts are ordered, we find that the original appropriation is insufficient.", 0],
    ["The allegations underlying this claim are that the County failed to maintain a swing at Fort Travis Park.", 0],
    ["Board Member Spencer - stated her opposition to the motion because she thinks it is important that the Board see all the numbers that are sent to the State of California, not just the cover page.", 0],
    ["The Committee member stated she is not comfortable with consolidation at this time.", 0],
    ["Councilmember Matarrese stated he is impressed with the results; the momentum needs to continue.", 1],
    ["Councilmember Tam stated that she is impressed with the North of Lincoln Avenue and Webster Street sales tax; inquired what triggered the increase.", 1],
    ["Councilmember Johnson stated staff has done an excellent job; the City needs to ensure that funding is preserved.", 1],
    ["Proponents: (In favor of ordinance): Robb Ratto, PSBA; and Sherri Stieg, WABA.", 1],
    ["Councilmember Matarrese stated that the Dancing Trees were great.", 1],
    ["APPROVED the recommended regulatory changes to the General Assistance program effective November 1, 2011, as recommended by the Director of Human Services", 1],
    ["Tourism brings so much money to your community.", 1],
    ["Think on this and we hope and pray that you all vote in favor of them because this is important to our community.", 1],
    ["And he’s been doing a great job in the six to seven months he’s been here.", 1],
    ["It's just pure joy.", 1],
    ["And I know we've got a good reporter behind us.", 1],
    ["These funds are valuable to aid our county in achieving a whole community concept of Emergency Management.", 1],
    ["Thank you for providing this much needed service to the older citizens of our district.", 1],
    ["I respectfully recommend approval by your Honorable Body.", 1],
    ["WHEREAS, Giggie Cortese is held by all who enjoy the pleasure of her acquaintance as a caring and compassionate woman, who generously gives of herself to the betterment of others, and has successfully balanced the demands of family, church, community, and vocation, and", 1],
    ["Our message is clear: People have a right to live and work, free from discrimination, regardless of their race, color, creed, gender or sexual orientation.", 1],
    ["Discussed his paddle board business; expressed support for the great water access the project will provide; urged approval: Mike Wong, Mike's Paddle.", 1],
    ["In our opinion, the information is fairly stated in all material respects, in relation to the financial statements as a whole.", 1],
    ["In those years he served with distinction and earned the respect and affection of his co-workers, many of whom also describe themselves as friends.", 1],
    ["Shelly Banks is a highly motivated manager and will be an asset to Cook County government.", 1],
    ["The existing proximity of housing to amenities in both cities, supported by accessible transportation, facilitates finding apartments that meet accessibility requirements.", 1],
    ["He was hopeful that everyone present would use transparency, creativity, honesty, and openness to work toward the common purpose.", 1],
]

SENTIMENT_TRAIN = [
    ["One student Committee member stated it is extremely difficult to get lunch on time during the time allotted.", 0],
    ["Joel Chew addressed liabilities regarding the Arena Hotel, where the City is providing housing to unhoused individuals, and discussed dangerous incidents that occurred at the Arena Hotel, such as a physical assault that he encountered.", 0],
    ["The appeal Was based on four issues: — that the subject parcel is a substandard illegally created lot, that the proposed dwelling exceeds County site standards for height and story limit, that the hazards posed by the coastal bluff were not adequately analyzed, and that the project should not have been certified as exempt from further environmental review under CEQA.",0],
    ["Vice Mayor Vella stated the bridge is a great regional opportunity; she looks forward to moving forward.", 1],
    ["Expressed support for the program, which is a great way to support those living in poverty: Bennett Schatz, Alameda.", 1],
    ["Mayor Johnson stated that ham operators are very enthusiastic to participate.", 1],
    ["WHEREAS, the Board of Supervisors has established an employee recognition program to publicly show support to employees for their outstanding efforts on behalf of the residents of the County of Santa Cruz; and", 1],
    ["I would certainly urge my colleagues to support this particular motion.", 1],
]

sentiment_training = pd.DataFrame(SENTIMENT_TRAIN, columns=["text", "label"])
sentiment_testing = pd.DataFrame(SENTIMENT_TEST, columns=["text", "label"])

In [ ]:
# sentiment_training = pd.read_csv("sentiment_training.csv", names=["text", "label"])
# sentiment_testing = pd.read_csv("sentiment_testing.csv", names=["text", "label"])


In [132]:
train_dataset = tf.data.Dataset.from_tensor_slices((sentiment_training["text"].values, sentiment_training["label"].values))
train_dataset.element_spec

test_dataset = tf.data.Dataset.from_tensor_slices((sentiment_testing["text"].values, sentiment_testing["label"].values))

In [120]:
for example, label in train_dataset.take(1):
  print('text: ', example.numpy())
  print('label: ', label.numpy())

text:  b'One student Committee member stated it is extremely difficult to get lunch on time during the time allotted.'
label:  0


In [133]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [134]:
for example, label in train_dataset.take(1):
  print('texts: ', example.numpy()[:3])
  print()
  print('labels: ', label.numpy()[:3])

texts:  [b'Expressed support for the program, which is a great way to support those living in poverty: Bennett Schatz, Alameda.'
 b'Vice Mayor Vella stated the bridge is a great regional opportunity; she looks forward to moving forward.'
 b'Mayor Johnson stated that ham operators are very enthusiastic to participate.']

labels:  [1 1 1]


In [135]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

In [136]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'the', 'to', 'that', 'is', 'support', 'of', 'and',
       'a', 'stated', 'on', 'for', 'time', 'program', 'not', 'mayor',
       'hotel', 'great', 'forward'], dtype='<U13')

In [137]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

array([[104,   6,  12,   2,  14,  25,   5,   9,  18,  29,   3,   6,  37,
         80,  89,  62, 133,  51, 144,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0],
       [ 31,  16,  33,  10,   2, 130,   5,   9,  18,  55,  69,  50,  79,
         19,   3,  74,  19,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0],
       [ 16,  83,  10,   4,  98,  70, 139,  32, 109,   3,  66,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0

In [138]:
for n in range(3):
  print("Original: ", example[n].numpy())
  print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
  print()

Original:  b'Expressed support for the program, which is a great way to support those living in poverty: Bennett Schatz, Alameda.'
Round-trip:  expressed support for the program which is a great way to support those living in poverty bennett schatz alameda                                         

Original:  b'Vice Mayor Vella stated the bridge is a great regional opportunity; she looks forward to moving forward.'
Round-trip:  vice mayor vella stated the bridge is a great regional opportunity she looks forward to moving forward                                           

Original:  b'Mayor Johnson stated that ham operators are very enthusiastic to participate.'
Round-trip:  mayor johnson stated that ham operators are very enthusiastic to participate                                                 



In [139]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='softplus'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

In [140]:
print([layer.supports_masking for layer in model.layers])

[False, True, True, True, True, True, True]


In [141]:
sample_text = "Chair Andersen adjourned the meeting at 10:40 AM."
predictions = model.predict(tf.constant([sample_text]))
print(predictions[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
[-0.083]


In [142]:
model.compile(loss=tf.keras.losses.KLDivergence(reduction="sum_over_batch_size"),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy', "recall", "precision"])

In [143]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 0.5000 - loss: 6.0780 - precision: 0.6667 - recall: 0.4000 - val_accuracy: 0.4500 - val_loss: 8.8650 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/10


/Users/phildini/civicband/sanitation/.venv/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 277ms/step - accuracy: 0.0000e+00 - loss: 8.1766 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.4500 - val_loss: 8.8650 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - accuracy: 0.1250 - loss: 10.0738 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.4500 - val_loss: 8.8650 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step - accuracy: 0.5000 - loss: 2.1403 - precision: 0.6000 - recall: 0.6000 - val_accuracy: 0.4500 - val_loss: 8.8650 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step - accuracy: 0.6250 - loss: 4.1853 - precision: 1.0000 - recall: 0.4000 - val_accuracy: 0.4500 - val_loss: 8.8650 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step - accuracy: 0.2500 - loss: 4.5556 - precision: 0.0000e+00 - recall: 0.0000e+